In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

%matplotlib inline 
#notebook을 실행한 브라우저에서 바로 시각화 자료를 볼수있게 함

titanic_df = pd.read_csv('/kaggle/input/titanic/train.csv') #같은 디렉토리안의 train.csv 파일을 df형식으로 읽어와서 객체 생성
titanic_df.head(3) #상위 3개의 row만 뽑아옴

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
print(titanic_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [4]:
titanic_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [5]:
print('결측지 처리전 데이터 세트 Null 값 갯수 \n',titanic_df.isnull().sum())

결측지 처리전 데이터 세트 Null 값 갯수 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [6]:
titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True) #inplace 파라미터 True ->적용한것을 바로 반환
titanic_df['Cabin'].fillna('N', inplace=True) 
titanic_df['Embarked'].fillna('N', inplace=True)
print('결측지 처리후 데이터 세트 Null 값 갯수 ',titanic_df.isnull().sum())

결측지 처리후 데이터 세트 Null 값 갯수  PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [7]:
# 학습 및 예측에  불필요한 피처들 제거
def drop_features(df):
    df.drop(['Name','Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 데이터 전처리 함수 정의
def transform_features(df):
    df = drop_features(df)
    df = format_features(df)
    return df

In [8]:
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived',axis=1, inplace=False)
X_titanic_df = transform_features(X_titanic_df) #전처리 적용
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, \
                                                  test_size=0.2, random_state=11)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 9) (179, 9) (712,) (179,)


In [9]:
X_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
431,432,3,0,29.699118,1,0,16.1000,7,3
821,822,3,1,27.000000,0,0,8.6625,7,3
629,630,3,1,29.699118,0,0,7.7333,7,2
626,627,2,1,57.000000,0,0,12.3500,7,2
665,666,2,1,32.000000,2,0,73.5000,7,3
...,...,...,...,...,...,...,...,...,...
638,639,3,0,41.000000,0,5,39.6875,7,3
771,772,3,1,48.000000,0,0,7.8542,7,3
521,522,3,1,22.000000,0,0,7.8958,7,3
711,712,1,1,29.699118,0,0,26.5500,2,3


In [10]:
test_data=pd.read_csv('/kaggle/input/titanic/test.csv')

test_data=transform_features(test_data)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True) #inplace 파라미터 True ->적용한것을 바로 반환
test_data['Cabin'].fillna('N', inplace=True) 
test_data['Embarked'].fillna('N', inplace=True)
test_data.dropna(inplace=True)
test_data.loc[418]=test_data.loc[417]
#test_data[:418]=test_data[:417]
test_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,892.0,3.0,1.0,34.50000,0.0,0.0,7.8292,7.0,1.0
1,893.0,3.0,0.0,47.00000,1.0,0.0,7.0000,7.0,2.0
2,894.0,2.0,1.0,62.00000,0.0,0.0,9.6875,7.0,1.0
3,895.0,3.0,1.0,27.00000,0.0,0.0,8.6625,7.0,2.0
4,896.0,3.0,0.0,22.00000,1.0,1.0,12.2875,7.0,2.0
...,...,...,...,...,...,...,...,...,...
414,1306.0,1.0,0.0,39.00000,0.0,0.0,108.9000,2.0,0.0
415,1307.0,3.0,1.0,38.50000,0.0,0.0,7.2500,7.0,2.0
416,1308.0,3.0,1.0,30.27259,0.0,0.0,8.0500,7.0,2.0
417,1309.0,3.0,1.0,30.27259,1.0,1.0,22.3583,7.0,0.0


In [11]:
# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train , y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train , y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도:{0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train , y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))


DecisionTreeClassifier 정확도: 0.7318
RandomForestClassifier 정확도:0.8492
LogisticRegression 정확도: 0.8603


In [12]:
scores = cross_val_score(dt_clf, X_titanic_df , y_titanic_df , cv=5)
for iter_count,accuracy in enumerate(scores):
    print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))

print("평균 정확도: {0:.4f}".format(np.mean(scores)))

교차 검증 0 정확도: 0.6034
교차 검증 1 정확도: 0.7809
교차 검증 2 정확도: 0.8090
교차 검증 3 정확도: 0.7921
교차 검증 4 정확도: 0.8202
평균 정확도: 0.7611


In [13]:
parameters = {'max_depth':[2,3,5,10],
             'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]} #파라미터 설정

grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5) #의사결정나무 모델로 GridSearchCV 적용

grid_dclf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터 :', grid_dclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_


# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
dpredictions = best_dclf.predict(test_data)
my_first_submission = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": dpredictions})
my_first_submission.drop(my_first_submission.index[417],inplace=True)
display(my_first_submission)

GridSearchCV 최적 하이퍼 파라미터 : {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.7978


,PassengerId,Survived
0,892.0,0
1,893.0,1
2,894.0,0
3,895.0,0
4,896.0,1
...,...,...
413,1305.0,0
414,1306.0,1
415,1307.0,0
416,1308.0,0


In [14]:
data = {'PassengerId': [1310],
         'Survived': [1]}

addition_df = pd.DataFrame(data)
my_first_submission = pd.concat([my_first_submission, addition_df], ignore_index=True)
my_first_submission = my_first_submission.astype({'PassengerId':'int'})
display(my_first_submission)
my_first_submission.to_csv("my_first_submission.csv", index=False)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1306,1
414,1307,0
415,1308,0
416,1309,0


In [15]:
dpredictions
len(dpredictions)
#test_prediction=best_dclf.predict(test_data)

418